### Importation of required librairy

In [15]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

### Dowload Dataset

In [16]:
url= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [17]:
# Create a dash application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [18]:
# Create an app layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36',
                                                    'font-size': 40}),
    html.Br(),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='ALL',
                 placeholder='Select a Launch Site here',
                 searchable=True
                 ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider',
                    min=0,
                    max=10000,
                    step=1000,
                    marks={0: '0',
                           1000: '1000',
                           2000: '2000',
                           3000: '3000',
                           4000: '4000',
                           5000: '5000',
                           6000: '6000',
                           7000: '7000',
                           8000: '8000',
                           9000: '9000',
                           10000: '10000'},
                    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]
                    ),
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    html.Br()
])

In [19]:
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_pie_chart(site):
    # Check if the selected site is 'ALL'
    if site == 'ALL':
        # Create a pie chart showing the total successful launches for all sites
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:
        # Filter the dataframe to include only the selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == site]
        # Create a pie chart showing the total successful launches for the selected site
        fig = px.pie(filtered_df, names='class', title=f'Total Success Launches for site {site}')
    # Return the figure to be displayed in the pie chart component
    return fig

**Explanation:**

- The @app.callback decorator defines the callback function that updates the pie chart.<br>
- The Output specifies that the callback will update the figure property of the component with ID success-pie-chart.
- The Input specifies that the callback will be triggered when the value property of the component with ID site-dropdown changes.
- The function update_pie_chart takes the selected site as input and generates a pie chart based on the selected site.
   - If 'ALL' is selected, the pie chart shows the total successful launches for all sites.
   - If a specific site is selected, the pie chart shows the total successful launches for that site.
- The generated figure is returned and displayed in the pie chart component.

In [20]:
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(site, payload):
    # Check if the selected site is 'ALL'
    if site == 'ALL':
        # Filter the dataframe to include only the payloads within the selected range
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
        # Create a scatter chart showing the correlation between payload and success for all sites
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Correlation between Payload and Success for all Sites')
    else:
        # Filter the dataframe to include only the selected launch site and payloads within the selected range
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1]) & (spacex_df['Launch Site'] == site)]
        # Create a scatter chart showing the correlation between payload and success for the selected site
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Correlation between Payload and Success for site {site}')
    # Return the figure to be displayed in the scatter chart component
    return fig

**Explanation:**

- The @app.callback decorator defines the callback function that updates the scatter chart.
- The Output specifies that the callback will update the figure property of the component with ID success-payload-scatter-chart.
- The Input specifies that the callback will be triggered when the value property of the components with IDs site-dropdown and payload-slider change.
- The function update_scatter_chart takes the selected site and payload range as input and generates a scatter chart based on the selected site and payload range.
  - If 'ALL' is selected, the scatter chart shows the correlation between payload and success for all sites within the selected payload range.
  - If a specific site is selected, the scatter chart shows the correlation between payload and success for that site within the selected payload range.
- The generated figure is returned and displayed in the scatter chart component.

**These callback functions ensure that the visualizations in the dashboard are dynamically updated based on user input, providing an interactive and informative experience.**

In [21]:
if __name__ == '__main__':
    app.run_server()